# Window Functions Summary

Window Functions are incredibly powerful parts of the SQL language and vastly under-utilised, possibly due t the syntax involved.

However, once the syntax is understood and the concepts grasped, they can become invaluable for reporting and analysis of data in a clean and powerful manner.

#### Queries

At the beginning of the section we were given several reporting requirements:

-   Report One
    -   Add an Identity column to our table as a whole ordered by order date then salesPersonID
    -   Secondary Identity for each employee ordered by saleAmount (largest to smallest)
-   Report Two
    -   Add a column providing a cumulative sales total per salesPersonID
-   Report Three
    -   Add a column showing the total saleAmount per salesPersonID
    -   Add columns showing the first and last saleAmount for each SalesPersonID

<u>Report One</u>

In order to add an incremental ID to our data we need to use a Ranking function.

As this is listed as an Indentity column this implies no duplication is possible and therefore we need a pure incremental seeding.  Hence <code>ROW_NUMBER()</code> which we will order by orderDate and salesPersonID.

We need a secondary identity column adding for each employee.  This implies another <code>ROW_NUMBER()</code>, but this time including the use of PARTITION BY.

Let's approach this in parts.

Firstly the Identity across the whole table, ordered by orderDate and SalesPersonID:

In [ ]:
select row_number() over(order by orderDate, salesPersonID) as myIdentityColumn, *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Now we shall tackle the second identity, ordering by saleAmount descending (as largest to smallest was specified) and partitioning by SalesPersonID:

In [ ]:
select row_number()
		over
		(
			partition by salesPersonID
			order by saleAmount desc
		) as mySecondaryIdentityColumn, *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Putting these together (taking the two row_number columns and placing them into a single query), gives us our report:

In [ ]:
select row_number() over(order by orderDate, salesPersonID) as myIdentityColumn,
		row_number()
		over
		(
			partition by salesPersonID
			order by saleAmount desc
		) as mySecondaryIdentityColumn, *
from JupyterDatabase.sales.salesOrderHeaderWindow
order by myIdentityColumn;

<u>Report Two</u>

In order to add a column showing cumulative saleAmount per salesPersonID we immediately think SUM.

Thinking back to our Aggregates we know that the default state of a Window Function is to process row by row through a table and therefore we do not need any explicit framing syntax to be used, instead we partition by salesPersonID, order by orderDate, and let SUM provide the running total:

In [ ]:
select *, sum(saleAmount)
            over
            (
                partition by salesPersonID
                order by salesPersonID, orderDate
            ) as runningSalesTotal
from JupyterDatabase.sales.salesOrderHeaderWindow;

<u>Report Three</u>

This report is a little more complicated as we need to accommodate the following:
* Add a column showing the total saleAmount per salesPersonID
* Add columns showing the first and last saleAmount for each SalesPersonID

Firstly, we have just used SUM in terms of a cumulative total but this is requiring us to use SUM based on a frame and therefore we will need to use our framing syntax:

In [ ]:
select *, sum(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as totalSales
from JupyterDatabase.sales.salesOrderHeaderWindow;

Additionally, we are required to show the first and last saleAmount for each salesPersonID.

Breaking this down, we obviously need <code>FIRST_VALUE</code> and <code>LAST_VALUE</code> but there is more to consider.  We can presume we need the sales value of the first order and last orders placed by each SalesPersonID and, as such, we are going to need to PARTITION BY salesPersonID and also ORDER BY orderDate.

For clarity we should also add framing into each of our functions.

This gives us the following:

In [ ]:
select *, first_value(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as firstSale,
			last_value(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as firstSale
from JupyterDatabase.sales.salesOrderHeaderWindow;

Combining these into our solution leaves us with the finished query:

In [ ]:
select *, first_value(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as firstSale,
			last_value(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as firstSale,
            sum(saleAmount)
			over
			(
				partition by salesPersonID
				order by orderDate
				rows between unbounded preceding and unbounded following
			) as totalSales
from JupyterDatabase.sales.salesOrderHeaderWindow;